데이터 불러오기
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

데이터 분할

In [3]:
import os
data_path='/content/drive/MyDrive/dataset/'
image_file_list=os.listdir(data_path+'images')
mask_file_list=os.listdir(data_path+'masks')

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_file_list,mask_file_list,test_size=0.2,shuffle=False,random_state=1004)
X_train,X_val,Y_train,Y_val=train_test_split(X_train,Y_train,test_size=0.2,shuffle=False,random_state=1004)

폴더 생성 함수

In [7]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

createFolder(data_path+'Original_Train')
createFolder(data_path+'Original_Test')
createFolder(data_path+'Original_val')
createFolder(data_path+'Mask_Train')
createFolder(data_path+'Mask_Test')
createFolder(data_path+'Mask_val')

생성된 각 폴더에 데이터 복사하기

In [8]:
import shutil

src=data_path+'images/'
dir=data_path+'Original_Train/'
for i in range(len(X_train)):
    shutil.copy(src+X_train[i],dir+X_train[i])

src=data_path+'masks/'
dir=data_path+'Mask_Train/'
for i in range(len(Y_train)):
    shutil.copy(src+Y_train[i],dir+Y_train[i])

src=data_path+'images/'
dir=data_path+'Original_val/'
for i in range(len(X_val)):
    shutil.copy(src+X_val[i],dir+X_val[i])
    
src=data_path+'masks/'
dir=data_path+'Mask_val/'
for i in range(len(Y_val)):
    shutil.copy(src+Y_val[i],dir+Y_val[i])

src=data_path+'images/'
dir=data_path+'Original_Test/'
for i in range(len(X_test)):
    shutil.copy(src+X_test[i],dir+X_test[i])

src=data_path+'masks/'
dir=data_path+'Mask_Test/'
for i in range(len(Y_test)):
    shutil.copy(src+Y_test[i],dir+Y_test[i])

generator 단위로 만들기

In [4]:
from keras.preprocessing.image import ImageDataGenerator
# we create two instances with the same arguments

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest',
                    rescale=1./255)

mask_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen  = ImageDataGenerator(**mask_gen_args) 

# Provide the same seed and keyword arguments to the fit and flow methods
#image_datagen.fit(images, augment=True, seed=seed)
#mask_datagen.fit(masks, augment=True, seed=seed)

**flow_from_directory**   
디렉토리에의 경로를 전달받아 증강된 데이터의 배치를 생성

In [9]:
###train
image_generator = image_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['Original_Train'],
    seed=1,
    batch_size=4,
    target_size=(256,256))

mask_generator = mask_datagen.flow_from_directory(
    data_path,
    classes=['Mask_Train'],
    class_mode=None,
    seed=1,
    color_mode='rgb',
    batch_size=4,
    target_size=(256,256))

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

###validation
val_image_generator = image_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['Original_val'],
    seed=1,
    batch_size=4,
    target_size=(256,256))

val_mask_generator = mask_datagen.flow_from_directory(
    data_path,
    classes=['Mask_val'],
    class_mode=None,
    seed=1,
    batch_size=4,
    color_mode='rgb',
    target_size=(256,256))

val_generator = zip(val_image_generator, val_mask_generator)

###test
test_image_generator=image_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['Original_Test'],
    seed=1,
    batch_size=4,
    color_mode='rgb',
    target_size=(256,256))

test_mask_generator=mask_datagen.flow_from_directory(
    data_path,
    class_mode=None,
    classes=['Mask_Test'],
    seed=1,
    batch_size=4,
    color_mode='rgb',
    target_size=(256,256))

test_generator=zip(test_image_generator,test_mask_generator)

Found 448 images belonging to 1 classes.
Found 448 images belonging to 1 classes.
Found 112 images belonging to 1 classes.
Found 112 images belonging to 1 classes.
Found 141 images belonging to 1 classes.
Found 141 images belonging to 1 classes.


colormap

In [12]:
import pandas as pd
classes = pd.read_csv(data_path+'class_dict.csv', index_col =0)
classes

,r,g,b
name,,,
Animal,64,128,64
Archway,192,0,128
Bicyclist,0,128,192
Bridge,0,128,64
Building,128,0,0
Car,64,0,128
CartLuggagePram,64,0,192
Child,192,128,64
Column_Pole,192,192,128


In [13]:
n_classes = len(classes)
n_classes

32

UNET 모델 정의
---
* Unet은 2015년에 출시한  Biomedical Image segmentation을 위한 convolutional network로 출시됨  
* End-to-End 로 Segmentation하는 심플하고 효과적인 방법  
* U-Net은 Fully Convolution Network(FCN)를 기반으로 하여 구축하였으며, 적은 데이터를 가지고도 더욱 정확한 Segmentaion을 내기 위해 FCN 구조를 수정


In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

def unet(n_classes, pretrained_weights = None,input_size = (256,256,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv10 = Conv2D(n_classes, (1,1), activation = 'softmax')(conv9)
    conv10 = Conv2D(n_classes, (1,1), padding='same')(conv9)  

    output_layer = conv10
    output_layer = Activation('softmax')(output_layer)

    model = Model( inputs,output_layer)
    
    #케라스의 MeanIOU를 평가지표로 사용
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = [tf.keras.metrics.MeanIoU(num_classes=32)])

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [ ]:
model = unet(n_classes)
model.summary()

이미지를 숫자형으로 변경
---

In [14]:
cls2rgb = {cl:list(classes.loc[cl, :]) for cl in classes.index}
cls2rgb

{'Animal': [64, 128, 64],
 'Archway': [192, 0, 128],
 'Bicyclist': [0, 128, 192],
 'Bridge': [0, 128, 64],
 'Building': [128, 0, 0],
 'Car': [64, 0, 128],
 'CartLuggagePram': [64, 0, 192],
 'Child': [192, 128, 64],
 'Column_Pole': [192, 192, 128],
 'Fence': [64, 64, 128],
 'LaneMkgsDriv': [128, 0, 192],
 'LaneMkgsNonDriv': [192, 0, 64],
 'Misc_Text': [128, 128, 64],
 'MotorcycleScooter': [192, 0, 192],
 'OtherMoving': [128, 64, 64],
 'ParkingBlock': [64, 192, 128],
 'Pedestrian': [64, 64, 0],
 'Road': [128, 64, 128],
 'RoadShoulder': [128, 128, 192],
 'SUVPickupTruck': [64, 128, 192],
 'Sidewalk': [0, 0, 192],
 'SignSymbol': [192, 128, 128],
 'Sky': [128, 128, 128],
 'TrafficCone': [0, 0, 64],
 'TrafficLight': [0, 64, 64],
 'Train': [192, 64, 128],
 'Tree': [128, 128, 0],
 'Truck_Bus': [192, 128, 192],
 'Tunnel': [64, 0, 64],
 'VegetationMisc': [192, 192, 0],
 'Void': [0, 0, 0],
 'Wall': [64, 192, 0]}

**adjust_mask(mask, flat)**

이미지를 숫자배열로 변환하는 역할


In [15]:
import numpy as np

def adjust_mask(mask, flat=False): 
  
    semantic_map = []
    for colour in list(cls2rgb.values()):        
        equality = np.equal(mask, colour) #256x256x3차원을 True/False로 채움
        class_map = np.all(equality, axis = -1) #256x256가 전부 True이면(지정된 픽셀의 색상을 찾으면) True 반환
        semantic_map.append(class_map) #True/False로 256x256 배열의 리스트 생성
    semantic_map = np.stack(semantic_map, axis=-1)# 256x256 배열 x32개 클래스 -> 256x256x32차원으로 만듬
    
    if flat:
        semantic_map = np.reshape(semantic_map, (-1,256*256)) #평면이 아니라면 (-1,256*256)의 2차원으로 변환

    return np.float32(semantic_map)#T/F 배열을 숫자배열로 변환

**generator_fn()**  
generator 내의 모든 값에 adjust_mask함수 적용

In [16]:
def train_generator_fn():
    for (img,mask) in train_generator:
        new_mask = adjust_mask(mask)
        yield (img,new_mask)

def val_generator_fn():
    for (img,mask) in val_generator:
        new_mask = adjust_mask(mask)
        yield (img,new_mask)

def test_generator_fn():  
    for (img,mask) in test_generator:
        new_mask=adjust_mask(mask)
        yield (img,new_mask)

모델 실행 및 시각화
---

학습, 검증, 평가 과정  진행

In [ ]:
model_checkpoint = ModelCheckpoint('unet_camvid.hdf5', monitor='val_loss',verbose=1, save_best_only=True)

model = unet(n_classes=32)
model.fit_generator(generator=train_generator_fn(),
                    validation_data=val_generator_fn(),
                    validation_steps=100,               
                    steps_per_epoch=1000,
                    epochs=10,
                    callbacks=[model_checkpoint])

In [ ]:
scores=model.evaluate_generator(test_generator_fn(),steps=5)
scores[1]

시각화

In [ ]:
# img (256,256,3)
# gt_mask: gt_mode=sparse--> (256,256) or ohe --> (256,256,32)
def test_visualize_seg(img, gt_mask, shape='normal'):
    plt.figure(1)

      # Img
    plt.subplot(311)
    plt.imshow(img)

      # Predict
    pred_mask = model.predict(np.expand_dims(img, 0))
    pred_mask = np.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[0]
    if shape=='flat':
        pred_mask = np.reshape(pred_mask, (256,256)) # Reshape only if you use the flat model. O.w. you dont need

    rgb_mask = np.apply_along_axis(map_class_to_rgb, -1, np.expand_dims(pred_mask, -1))

    # Prediction
    plt.subplot(312)
    plt.imshow(rgb_mask)

    plt.subplot(313)
    plt.imshow((gt_mask).astype(np.float64))

In [ ]:
img = next(test_image_generator)[0]
gt_img = next(test_mask_generator)[0]
test_visualize_seg(img, gt_img)